Model is the lactose pathway. lactose is passively diffused through porin pores, active uptake by lac Permease, and then hydrolysis by beta-Galactosidase. 

In [2]:
import tellurium as te
print ("success")

success


Make sure you understand what you're plotting. The fundamental variable that gets changed is time because this is a kinetic modeling package.

Instead of doing the model within a python cell, it's better to use a model cell and get in the habit of using that. It's easier to not have to reload the entire model to plot or do any manipulations. It also allows you to order stuff in whatever order. It cross checks to make sure all the variables are there so you don't have to work completely linearly.

-> means irreversible

=> expect reversible rate law.

you can calculate the steady state values 
values = LacPath.getSteadyStateValues()
selections = LacPath.steadyStateSelections

print(values)

print(selections)

Because in a tiny cell, the concentrations are small, they'll perturb quickly. So steady state will occur very rapidly. So we'll mostly worry about what happens at steady state. 


In [ ]:
model 
    compartment C2, C3;
    
    species S3 in C3
    
    J34: S3 -> ; kcat_34*Etot_34*S3 / (Km_34 + S3)
    kcat_34 = 6.42e2; #S^-1; from Juers et al (2012)
    Etot_34 = 1.0e-3 ; #M, free parameter
    Km_34 = 1.01e1 ; # M; from BRENDA 5.5e-3
    
    S3 = 100;
    
    end
    

In [4]:
import math
import numpy as np
import tellurium as te
# this is our calculation to check our numerical calculations
# 1 liter = 1 dm^3
area = 1.0e-10; # dm^2
permeability = 1.0e-8; # dm s-1
km = 5.5e-3; # M
S1 = 10e-3; # M
kcat = 6.42e2; # s-1
etot = 1.0e-6; # M
vol_cell = 1.0e-15; # L or dm^3
a = area * permeability;
b = km*area*permeability-area*permeability*S1+kcat*etot*vol_cell;
c = -1*km*area*permeability*S1;
S3_ss=(-b+(b**2-4*a*c)**0.5)/(2*a);
print(a,b,c)
print('S3_ss',S3_ss)

1e-18 6.375000000000001e-19 -5.5e-23
S3_ss 8.626283721240081e-05


In [27]:
model LacPath
    compartment inside_cell;
    inside_cell = 1.0e-6;
    compartment outside_cell;
    outside_cell = 1.0;
    species lactose, glucgalac
    const species outside_lactose;
    unit M = mole / liter
    unit inv_sec = 1. / seconds
    unit vol_conv = 1 liter / 1 dm
    unit um = 10.e-6 meters
    unit dm = 10.e-2 meters
    
    lactose = 0.00 M; # M; free parameter
    glucgalac = 0.0 M;
    outside_lactose = 10e-3 M;
    vol_cell = 1.0e-15; # L or dm^3

    lactose in inside_cell; glucgalac in inside_cell;
    outside_lactose in outside_cell
    
    J34: lactose -> ; (kcat*Etot*lactose)/(Km+lactose)*vol_cell;

    kcat = 6.42e2 ; #s^-1; from Juers et al (2012) 
    Etot = 1.e-6 ; # M; free parameter
    Km = 5.5e-3 ; # M; from BRENDA
    
    J13: outside_lactose => lactose; area_cell*permeability*(outside_lactose-lactose); #this is already the net flux so you don't need a back reaction. Jnet = Jin - Jout. So, has to permeability*area (lactose external - 0)- permeability*area(Lactose internal - 0) works for simple diffusion
    area_cell = 1e-10 
    permeability = 1e-8 ;
end

#r=te.loada(model)



Success: Model can be accessed via variable LacPath


In [28]:
LacPath.reset()
LacPath.simulate(0., 10e10, 1000) #'S1','J1'
LacPath.plot()

LacPath.steadyStateSelections = ['[lactose]',"J34"]
values = LacPath.getSteadyStateValues()
selections = LacPath.steadyStateSelections
print(values)
print(selections)

[  8.62628097e-05   9.91373405e-21]
['[lactose]', 'J34']
